<a href="https://colab.research.google.com/github/purin96/Data-Science-Projects-2024/blob/main/pima_classification_baseline_05ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
df=pd.read_csv("/content/sample_data/diabetes_feature.csv")
df.shape

(768, 16)

In [3]:
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome,Pregnancies_high,Age_low,Age_middle,Age_high,Insulin_nan,Insulin_log,low_glu_insulin
0,6,148,72,35,0,33.6,0.627,50,1,False,False,True,False,169.5,5.138735,False
1,1,85,66,29,0,26.6,0.351,31,0,False,False,True,False,102.5,4.639572,True
2,8,183,64,0,0,23.3,0.672,32,1,True,False,True,False,169.5,5.138735,False
3,1,89,66,23,94,28.1,0.167,21,0,False,True,False,False,94.0,4.553877,True
4,0,137,40,35,168,43.1,2.288,33,1,False,False,True,False,168.0,5.129899,False


# 학습과 예측에 사용될 데이터

In [4]:
df.columns

Index(['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin',
       'BMI', 'DiabetesPedigreeFunction', 'Age', 'Outcome', 'Pregnancies_high',
       'Age_low', 'Age_middle', 'Age_high', 'Insulin_nan', 'Insulin_log',
       'low_glu_insulin'],
      dtype='object')

In [5]:
X = df[['Glucose', 'BloodPressure', 'SkinThickness',
       'BMI', 'DiabetesPedigreeFunction', 'Age',
     'Pregnancies_high',
       'Insulin_nan',
       'low_glu_insulin']]
X.shape

(768, 9)

In [6]:
y=df["Outcome"]
y.shape

(768,)

# 학습과 예측에 사용할 데이터셋 만들기

In [7]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
X, y, test_size=0.2, random_state=42)

In [8]:
X_train.shape, y_train.shape

((614, 9), (614,))

In [9]:
X_test.shape,y_test.shape

((154, 9), (154,))

# 학습하기

In [10]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier

estimators =[ DecisionTreeClassifier(random_state=42),
              RandomForestClassifier(random_state=42),
              GradientBoostingClassifier(random_state=42)]
estimators

[DecisionTreeClassifier(random_state=42),
 RandomForestClassifier(random_state=42),
 GradientBoostingClassifier(random_state=42)]

In [11]:
results = []
for estimator in estimators:
  result = []
  result.append(estimator.__class__.__name__)
  results.append(result)
results

[['DecisionTreeClassifier'],
 ['RandomForestClassifier'],
 ['GradientBoostingClassifier']]

In [12]:
max_depth = np.random.randint(2,20,10)
max_depth

array([ 8,  8, 17,  6, 10,  2, 18,  9, 17,  3])

In [13]:
max_features = np.random.uniform(0.3,1,10)
max_features

array([0.79775079, 0.64006279, 0.50342251, 0.48242169, 0.79341364,
       0.5863765 , 0.84860734, 0.77873374, 0.8988582 , 0.7301705 ])

In [14]:
from sklearn.model_selection import RandomizedSearchCV
max_depth = np.random.randint(2,20,10)
max_features = np.random.uniform(0.3,1,10)
param_distributions = {"max_depth":max_depth,
                       "max_features":max_features}
results = []
for estimator in estimators:
  result = []
  if estimator.__class__.__name__ !="DecisionTreeClassifier":
     param_distributions["n_estimators"]= np.random.randint(100,200,10)
  clf = RandomizedSearchCV(estimator,
                    param_distributions,
                    n_iter =100,
                    scoring ="accuracy",
                    n_jobs=-1,
                    cv=5,
                    verbose=2 )
  clf.fit(X_train,y_train)
  result.append(estimator.__class__.__name__)
  result.append(clf.score(X_test,y_test))
  result.append(clf.best_params_)
  result.append(clf.best_score_)
  result.append(clf.cv_results_)
  results.append(result)

Fitting 5 folds for each of 100 candidates, totalling 500 fits
Fitting 5 folds for each of 100 candidates, totalling 500 fits
Fitting 5 folds for each of 100 candidates, totalling 500 fits


In [23]:
df = pd.DataFrame(results,columns=["estimator","test_score","best_params","train_score","cv_results_"])
df

,estimator,test_score,best_params,train_score,cv_results_
0,DecisionTreeClassifier,0.876623,"{'max_features': 0.861457417312145, 'max_depth...",0.868106,"{'mean_fit_time': [0.017292547225952148, 0.024..."
1,RandomForestClassifier,0.857143,"{'n_estimators': 107, 'max_features': 0.650357...",0.905611,"{'mean_fit_time': [0.6488945960998536, 0.49165..."
2,GradientBoostingClassifier,0.863636,"{'n_estimators': 173, 'max_features': 0.630377...",0.907210,"{'mean_fit_time': [1.0188422203063965, 0.96515..."


In [29]:
pd.DataFrame(df.loc[1,"cv_results_"]).sort_values(by="rank_test_score")

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_n_estimators,param_max_features,param_max_depth,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
43,0.354172,0.008347,0.014763,0.000517,107,0.650357,16,"{'n_estimators': 107, 'max_features': 0.650357...",0.886179,0.934959,0.861789,0.894309,0.950820,0.905611,0.032652,1
99,0.375596,0.051751,0.014864,0.002784,120,0.650357,11,"{'n_estimators': 120, 'max_features': 0.650357...",0.886179,0.934959,0.853659,0.894309,0.950820,0.903985,0.034918,2
59,0.356552,0.011084,0.014551,0.001023,107,0.630378,11,"{'n_estimators': 107, 'max_features': 0.630377...",0.886179,0.934959,0.861789,0.894309,0.942623,0.903972,0.030475,3
33,0.364576,0.010030,0.015064,0.001339,109,0.650357,17,"{'n_estimators': 109, 'max_features': 0.650357...",0.886179,0.934959,0.861789,0.894309,0.942623,0.903972,0.030475,3
8,0.584472,0.105918,0.021956,0.006334,109,0.616904,17,"{'n_estimators': 109, 'max_features': 0.616904...",0.886179,0.934959,0.861789,0.894309,0.942623,0.903972,0.030475,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67,0.300131,0.008035,0.013930,0.000393,109,0.616904,2,"{'n_estimators': 109, 'max_features': 0.616904...",0.796748,0.878049,0.853659,0.813008,0.934426,0.855178,0.048986,95
30,0.303767,0.013215,0.013932,0.000407,109,0.633874,2,"{'n_estimators': 109, 'max_features': 0.633873...",0.796748,0.878049,0.853659,0.813008,0.934426,0.855178,0.048986,95
74,0.485722,0.023535,0.021934,0.002440,170,0.75484,2,"{'n_estimators': 170, 'max_features': 0.754840...",0.796748,0.869919,0.845528,0.804878,0.942623,0.851939,0.052628,98
86,0.565573,0.012638,0.022646,0.000518,199,0.861457,2,"{'n_estimators': 199, 'max_features': 0.861457...",0.788618,0.869919,0.837398,0.804878,0.942623,0.848687,0.054644,99


In [19]:
clf.best_params_

{'n_estimators': 173, 'max_features': 0.6303776517610893, 'max_depth': 10}

In [20]:
clf.best_score_

0.9072104491536719